# Unit Testing `GiRaFFE_NRPy`: Fluxes of $\tilde{S}_i$

### Author: Patrick Nelson

This notebook validates our new, NRPyfied HLLE solver against the function from the original `GiRaFFE` that calculates the flux for $\tilde{S}_i$ according to the the method of Harten, Lax, von Leer, and Einfeldt (HLLE), assuming that we have calculated the values of the flux on the cell faces according to the piecewise-parabolic method (PPM) of [Colella and Woodward (1984)](https://crd.lbl.gov/assets/pubs_presos/AMCS/ANAG/A141984.pdf), modified for the case of GRFFE. 

**Module Status:** <font color=green><b> Validated: </b></font> This code has passed unit tests against the original `GiRaFFE` version.

**Validation Notes:** This demonstrates the validation of [Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux](../Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux.ipynb).

It is, in general, good coding practice to unit test functions individually to verify that they produce the expected and intended output. Here, we expect our functions `GRFFE__S_i__flux_in_dir_*.h` to produce identical output to the function `GRFFE__S_i__flux.C` in the original `GiRaFFE`. It should be noted that the two codes handle the parameter `flux_dirn` (the direction in which the code is presently calculating the flux through the cell) differently; in the original `GiRaFFE`, the function `GRFFE__S_i__flux()` expects a parameter `flux_dirn` with value 1, 2, or 3, corresponding to the functions `GRFFE__S_i__flux_in_dir_0()`, `GRFFE__S_i__flux_in_dir_1()`, and `GRFFE__S_i__flux_in_dir_2()`, respectively, in `GiRaFFE_NRPy`.

### NRPy+ Source Code for this module: 
* [GiRaFFE_NRPy/GiRaFFE_NRPy_Stilde_flux.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_Stilde_flux.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-Stilde-flux.ipynb) Generates the code to compute the fluxes.

## Introduction:

This notebook validates the NRPyfied C2P and P2C solvers against the original `GiRaFFE` code. This will be done at a point with a random but realistic spacetime and a variety of Poynting fluxes and Valencia velocities to test edge cases.

We'll write this in C because the codes we want to test are already written that way, and we would like to avoid modifying the files as much as possible. To do so, we will print the C code to a file. We will begin by including core functionality. We will also define standard parameters needed for GRFFE and NRPy+.

When this notebook is run, the significant digits of agreement between the old `GiRaFFE` and new `GiRaFFE_NRPy` versions of the algorithm will be printed to the screen right after the code is run [here](#compile_run).


<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#setup): Set up core functions and parameters for unit testing the C2P and P2C algorithms
    1. [Step 1.a](#c_flux) Write the C functions to compute the flux
    1. [Step 1.b](#free_params) Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h`
    1. [Step 1.c](#download) Download files from `GiRaFFE` for comparison
1. [Step 2](#mainc): `Stilde_flux_unit_test.c`: The Main C Code
    1. [Step 2.a](#compile_run): Compile and run the code to validate the output
1. [Step 3](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='setup'></a>

# Step 1: Set up core functions and parameters for unit testing the C2P and P2C algorithms \[Back to [top](#toc)\]

$$\label{setup}$$

We'll start by appending the relevant paths to `sys.path` so that we can access sympy modules in other places. Then, we'll import NRPy+ core functionality and set up a directory in which to carry out our test. We will also declare the gridfunctions that are needed for this portion of the code.

In [1]:
import os,sys
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)


The last things NRPy+ will require are the definition of type `REAL` and, of course, the functions we are testing. These files are generated on the fly.

In [2]:
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import outCfunction, lhrh # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

out_dir  = "Validation"
cmd.mkdir(out_dir)

thismodule = "Start_to_Finish_UnitTest-GiRaFFE_NRPy-Stilde_flux"

import GiRaFFE_NRPy.Stilde_flux as Sf

# We will pass values of the gridfunction on the cell faces into the function. This requires us
# to declare them as C parameters in NRPy+. We will denote this with the _face infix/suffix.
alpha_face,gammadet_face = gri.register_gridfunctions("AUXEVOL",["alpha_face","gammadet_face"])
gamma_faceDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gamma_faceDD","sym01")
gamma_faceUU = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gamma_faceUU","sym01")
beta_faceU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","beta_faceU")

# We'll need some more gridfunctions, now, to represent the reconstructions of BU and ValenciavU
# on the right and left faces
Valenciav_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","Valenciav_rU",DIM=3)
B_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_rU",DIM=3)
Valenciav_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","Valenciav_lU",DIM=3)
B_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_lU",DIM=3)

sqrt4pi = par.Cparameters("REAL",thismodule,"sqrt4pi","sqrt(4.0*M_PI)")

# And the function to which we'll write the output data:
Stilde_fluxD = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","Stilde_fluxD",DIM=3)


<a id='c_flux'></a>

## Step 1.a: Write the C functions to compute the flux \[Back to [top](#toc)\]

$$\label{c_flux}$$
Here, we will write out each of the three files that we need to generate to calculate the flux of $\tilde{S}_i$ in NRPy+.

In [3]:
# And now, we'll write the files
# In practice, the C functions should only loop over the interior; here, however, we can loop over
# all points and set fewer parameters.
for flux_dirn in range(3):
    desc="Compute the flux of all 3 components of tilde{S}_i in the "+str(flux_dirn)+"th direction"
    name="GRFFE__S_i__flux_in_dir_"+str(flux_dirn)
    Sf.calculate_Stilde_flux(flux_dirn,True,alpha_face,gamma_faceDD,beta_faceU,\
                             Valenciav_rU,B_rU,Valenciav_lU,B_lU,sqrt4pi)
    Stilde_flux_to_print   = [\
                              lhrh(lhs=gri.gfaccess("auxevol_gfs","Stilde_fluxD0"),rhs=Sf.Stilde_fluxD[0]),\
                              lhrh(lhs=gri.gfaccess("auxevol_gfs","Stilde_fluxD1"),rhs=Sf.Stilde_fluxD[1]),\
                              lhrh(lhs=gri.gfaccess("auxevol_gfs","Stilde_fluxD2"),rhs=Sf.Stilde_fluxD[2]),\
                             ]

    outCfunction(
        outfile  = os.path.join(out_dir,name+".h"), desc=desc, name=name,
        params   ="const paramstruct *params,REAL *auxevol_gfs",
        body     =fin.FD_outputC("returnstring",Stilde_flux_to_print,\
                                 params="outCverbose=False").replace("IDX4","IDX4S"),
        loopopts ="AllPoints")



Output C function GRFFE__S_i__flux_in_dir_0() to file Validation/GRFFE__S_i__flux_in_dir_0.h
Output C function GRFFE__S_i__flux_in_dir_1() to file Validation/GRFFE__S_i__flux_in_dir_1.h
Output C function GRFFE__S_i__flux_in_dir_2() to file Validation/GRFFE__S_i__flux_in_dir_2.h


<a id='free_params'></a>

## Step 1.b: Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h` \[Back to [top](#toc)\]

$$\label{free_params}$$

Based on declared NRPy+ Cparameters, first we generate `declare_Cparameters_struct.h`, `set_Cparameters_default.h`, and `set_Cparameters[-SIMD].h`.

Then we output `free_parameters.h`, which sets some basic grid parameters as well as the speed limit parameter we need for this function.

In [4]:
# Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
# par.generate_Cparameters_Ccodes(os.path.join(out_dir))

# Step 3.d.ii: Set free_parameters.h
with open(os.path.join(out_dir,"free_parameters.h"),"w") as file:
    file.write("""
// Set free-parameter values.
params.Nxx_plus_2NGHOSTS0 = 1;
params.Nxx_plus_2NGHOSTS1 = 1;
params.Nxx_plus_2NGHOSTS2 = 1;

// Standard GRFFE parameters:
params.GAMMA_SPEED_LIMIT = 2000.0;

\n""")

# Generates declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(out_dir))

<a id='download'></a>

## Step 1.c: Download files from `GiRaFFE` for comparison \[Back to [top](#toc)\]

$$\label{download}$$

We'll also need to download the files in question from the `GiRaFFE` bitbucket repository. This code was originally written by Leo Werneck in the IllinoisGRMHD documentation; we have modified it to download the files we want. Of note is the addition of the `for` loop since we need three files (The function `GRFFE__S_i__flux()` depends on two other files for headers and functions).

In [5]:
# First download the original IllinoisGRMHD source code
import urllib
original_file_url  = ["https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/GiRaFFE/src/GiRaFFE_headers.h",\
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/GiRaFFE/src/inlined_functions.C",\
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/GiRaFFE/src/GRFFE__S_i__flux.C"\
                     ]
original_file_name = ["GiRaFFE_headers.h",\
                      "inlined_functions.C",\
                      "GRFFE__S_i__flux.C"\
                     ]

for i in range(len(original_file_url)):
    original_file_path = os.path.join(out_dir,original_file_name[i])

    # Then download the original IllinoisGRMHD source code
    # We try it here in a couple of ways in an attempt to keep
    # the code more portable
    try:
        original_file_code = urllib.request.urlopen(original_file_url[i]).read().decode('utf-8')
    except:
        original_file_code = urllib.urlopen(original_file_url[i]).read().decode('utf-8')

    # Write down the file the original IllinoisGRMHD source code
    with open(original_file_path,"w") as file:
        file.write(original_file_code)


<a id='mainc'></a>

# Step 2: `Stilde_flux_unit_test.c`: The Main C Code \[Back to [top](#toc)\]

$$\label{mainc}$$

Now that we have our vector potential and analytic magnetic field to compare against, we will start writing our unit test. We'll also import common C functionality, define `REAL`, the number of ghost zones, and the faces, and set the standard macros for NRPy+ style memory access.Now we can write our C code. First, we will import our usual libraries and define the various constants and macros we need, taking care to imitate CCTK functionality wherever necessary. In the main function, we will fill all relevant arrays with (appropriate) random values. That is, if a certain gridfunction should never be negative, we will make sure to only generate positive numbers for it. We must also contend with the fact that in NRPy+, we chose to use the Valencia 3-velocity $v^i_{(n)}$, while in ETK, we used  the drift velocity $v^i$; the two are related by $$v^i = \alpha v^i_{(n)} - \beta^i.$$

In [13]:
%%writefile $out_dir/Stilde_flux_unit_test.C
// These are common packages that we are likely to need.
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "string.h" // Needed for strncmp, etc.
#include "stdint.h" // Needed for Windows GCC 6.x compatibility
#include <time.h>   // Needed to set a random seed.

// Standard GRFFE parameters:
const double GAMMA_SPEED_LIMIT = 2000.0;

// Standard NRPy+ memory access:
#define IDX4S(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) + Nxx_plus_2NGHOSTS2 * (g) ) ) )

// Let's also #define the NRPy+ gridfunctions
#define ALPHA_FACEGF 0
#define GAMMADET_FACEGF 1
#define GAMMA_FACEDD00GF 2
#define GAMMA_FACEDD01GF 3
#define GAMMA_FACEDD02GF 4
#define GAMMA_FACEDD11GF 5
#define GAMMA_FACEDD12GF 6
#define GAMMA_FACEDD22GF 7
#define GAMMA_FACEUU00GF 8
#define GAMMA_FACEUU11GF 9
#define GAMMA_FACEUU22GF 10
#define BETA_FACEU0GF 11
#define BETA_FACEU1GF 12
#define BETA_FACEU2GF 13
#define VALENCIAV_RU0GF 14
#define VALENCIAV_RU1GF 15
#define VALENCIAV_RU2GF 16
#define B_RU0GF 17
#define B_RU1GF 18
#define B_RU2GF 19
#define VALENCIAV_LU0GF 20
#define VALENCIAV_LU1GF 21
#define VALENCIAV_LU2GF 22
#define B_LU0GF 23
#define B_LU1GF 24
#define B_LU2GF 25
#define U4UPPERZERO_LGF 26
#define U4UPPERZERO_RGF 27
#define STILDE_FLUXD0GF 28
#define STILDE_FLUXD1GF 29
#define STILDE_FLUXD2GF 30
#define NUM_AUXEVOL_GFS 31

// The NRPy+ versions of the function. These should require relatively little modification.
// We will need this define, though:
#define REAL double
#include "declare_Cparameters_struct.h"

#include "GRFFE__S_i__flux_in_dir_0.h"
#include "GRFFE__S_i__flux_in_dir_1.h"
#include "GRFFE__S_i__flux_in_dir_2.h"

// Some needed workarounds to get the ETK version of the code to work
#define CCTK_REAL double
#define DECLARE_CCTK_PARAMETERS //
struct cGH{};
const cGH cctkGH;

// And include the code we want to test against
#include "GiRaFFE_headers.h"
#include "inlined_functions.C"
#include "GRFFE__S_i__flux.C"

int main() {
    paramstruct params;
#include "set_Cparameters_default.h"

    // Step 0c: Set free parameters, overwriting Cparameters defaults
    //          by hand or with command-line input, as desired.
#include "free_parameters.h"
#include "set_Cparameters-nopointer.h"

    // We'll define all indices to be 0. No need to complicate memory access
    const int i0 = 0;
    const int i1 = 0;
    const int i2 = 0;

    // This is the array to which we'll write the NRPy+ variables.
    REAL *auxevol_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS);

    // These are the arrays to which we will write the ETK variables.
    CCTK_REAL METRIC_LAP_PSI4[NUMVARS_METRIC_AUX];
    CCTK_REAL Ur[MAXNUMVARS];
    CCTK_REAL Ul[MAXNUMVARS];
    CCTK_REAL FACEVAL[NUMVARS_FOR_METRIC_FACEVALS];
    CCTK_REAL cmax; CCTK_REAL cmin;
    CCTK_REAL st_x_flux; CCTK_REAL st_y_flux; CCTK_REAL st_z_flux;

    // Now, it's time to make the random numbers.
    //const long int seed = time(NULL); // seed = 1570632212; is an example of a seed that produces
                                      // bad agreement for high speeds
    //const long int seed = 1574393335;
    //srand(seed); // Set the seed
    //printf("seed for random number generator = %ld; RECORD IF AGREEMENT IS BAD\n\n",seed);
    // We take care to make sure the corresponding quantities have the SAME value.
    auxevol_gfs[IDX4S(ALPHA_FACEGF, i0,i1,i2)] = 1.0;//(double)rand()/RAND_MAX;
    const double alpha = auxevol_gfs[IDX4S(ALPHA_FACEGF, i0,i1,i2)];
    METRIC_LAP_PSI4[LAPSE] = alpha;
    //METRIC_LAP_PSI4[LAPM1] = METRIC_LAP_PSI4[LAPSE]-1;

    auxevol_gfs[IDX4S(GAMMA_FACEDD00GF, i0,i1,i2)] = 1.0;//1.0+(double)rand()/RAND_MAX*0.2-0.1;
    auxevol_gfs[IDX4S(GAMMA_FACEDD01GF, i0,i1,i2)] = 0.0;//(double)rand()/RAND_MAX*0.2-0.1;
    auxevol_gfs[IDX4S(GAMMA_FACEDD02GF, i0,i1,i2)] = 0.0;//(double)rand()/RAND_MAX*0.2-0.1;
    auxevol_gfs[IDX4S(GAMMA_FACEDD11GF, i0,i1,i2)] = 1.0;//1.0+(double)rand()/RAND_MAX*0.2-0.1;
    auxevol_gfs[IDX4S(GAMMA_FACEDD12GF, i0,i1,i2)] = 0.0;//(double)rand()/RAND_MAX*0.2-0.1;
    auxevol_gfs[IDX4S(GAMMA_FACEDD22GF, i0,i1,i2)] = 1.0;//1.0+(double)rand()/RAND_MAX*0.2-0.1;

    // Generated by NRPy+:
       const double gammaDD00 = auxevol_gfs[IDX4S(GAMMA_FACEDD00GF, i0,i1,i2)];
       const double gammaDD01 = auxevol_gfs[IDX4S(GAMMA_FACEDD01GF, i0,i1,i2)];
       const double gammaDD02 = auxevol_gfs[IDX4S(GAMMA_FACEDD02GF, i0,i1,i2)];
       const double gammaDD11 = auxevol_gfs[IDX4S(GAMMA_FACEDD11GF, i0,i1,i2)];
       const double gammaDD12 = auxevol_gfs[IDX4S(GAMMA_FACEDD12GF, i0,i1,i2)];
       const double gammaDD22 = auxevol_gfs[IDX4S(GAMMA_FACEDD22GF, i0,i1,i2)];
       /*
        * NRPy+ Finite Difference Code Generation, Step 2 of 1: Evaluate SymPy expressions and write to main memory:
        */
       const double tmp0 = gammaDD11*gammaDD22;
       const double tmp1 = pow(gammaDD12, 2);
       const double tmp2 = gammaDD02*gammaDD12;
       const double tmp3 = pow(gammaDD01, 2);
       const double tmp4 = pow(gammaDD02, 2);
       const double tmp5 = gammaDD00*tmp0 - gammaDD00*tmp1 + 2*gammaDD01*tmp2 - gammaDD11*tmp4 - gammaDD22*tmp3;
       const double tmp6 = 1.0/tmp5;
       auxevol_gfs[IDX4S(GAMMA_FACEUU00GF, i0, i1, i2)] = tmp6*(tmp0 - tmp1);
       auxevol_gfs[IDX4S(GAMMA_FACEUU11GF, i0, i1, i2)] = tmp6*(gammaDD00*gammaDD22 - tmp4);
       auxevol_gfs[IDX4S(GAMMA_FACEUU22GF, i0, i1, i2)] = tmp6*(gammaDD00*gammaDD11 - tmp3);
       auxevol_gfs[IDX4S(GAMMADET_FACEGF, i0, i1, i2)] = tmp5;

    auxevol_gfs[IDX4S(GAMMADET_FACEGF, i0,i1,i2)] = tmp5;
    METRIC_LAP_PSI4[PSI6] = sqrt(auxevol_gfs[IDX4S(GAMMADET_FACEGF, i0,i1,i2)]);
    METRIC_LAP_PSI4[PSI2] = pow(METRIC_LAP_PSI4[PSI6],1.0/3.0);
    METRIC_LAP_PSI4[PSI4] = METRIC_LAP_PSI4[PSI2]*METRIC_LAP_PSI4[PSI2];
    const double Psim4 = 1.0/METRIC_LAP_PSI4[PSI4];
    METRIC_LAP_PSI4[PSIM4] = Psim4;

    // Copied from the ETK implementation
        CCTK_REAL gtxxL = gammaDD00*Psim4;
        CCTK_REAL gtxyL = gammaDD01*Psim4;
        CCTK_REAL gtxzL = gammaDD02*Psim4;
        CCTK_REAL gtyyL = gammaDD11*Psim4;
        CCTK_REAL gtyzL = gammaDD12*Psim4;
        CCTK_REAL gtzzL = gammaDD22*Psim4;

        /*********************************
         * Apply det gtij = 1 constraint *
         *********************************/
        const CCTK_REAL gtijdet = gtxxL * gtyyL * gtzzL + gtxyL * gtyzL * gtxzL + gtxzL * gtxyL * gtyzL -
          gtxzL * gtyyL * gtxzL - gtxyL * gtxyL * gtzzL - gtxxL * gtyzL * gtyzL;

        /*const CCTK_REAL gtijdet_Fm1o3 = fabs(1.0/cbrt(gtijdet));

        gtxxL = gtxxL * gtijdet_Fm1o3;
        gtxyL = gtxyL * gtijdet_Fm1o3;
        gtxzL = gtxzL * gtijdet_Fm1o3;
        gtyyL = gtyyL * gtijdet_Fm1o3;
        gtyzL = gtyzL * gtijdet_Fm1o3;
        gtzzL = gtzzL * gtijdet_Fm1o3;*/

    FACEVAL[GXX] = gtxxL;
    FACEVAL[GXY] = gtxyL;
    FACEVAL[GXZ] = gtxzL;
    FACEVAL[GYY] = gtyyL;
    FACEVAL[GYZ] = gtyzL;
    FACEVAL[GZZ] = gtzzL;

    FACEVAL[GUPXX] = ( gtyyL * gtzzL - gtyzL * gtyzL )/gtijdet;
    FACEVAL[GUPYY] = ( gtxxL * gtzzL - gtxzL * gtxzL )/gtijdet;
    FACEVAL[GUPZZ] = ( gtxxL * gtyyL - gtxyL * gtxyL )/gtijdet;

    auxevol_gfs[IDX4S(BETA_FACEU0GF, i0,i1,i2)] = 0.0;//(double)rand()/RAND_MAX*2.0-1.0;
    const double betax = auxevol_gfs[IDX4S(BETA_FACEU0GF, i0,i1,i2)];
    FACEVAL[SHIFTX] = betax;
    auxevol_gfs[IDX4S(BETA_FACEU1GF, i0,i1,i2)] = 0.0;//(double)rand()/RAND_MAX*2.0-1.0;
    const double betay = auxevol_gfs[IDX4S(BETA_FACEU1GF, i0,i1,i2)];
    FACEVAL[SHIFTY] = betay;
    auxevol_gfs[IDX4S(BETA_FACEU2GF, i0,i1,i2)] = 0.0;//(double)rand()/RAND_MAX*2.0-1.0;
    const double betaz = auxevol_gfs[IDX4S(BETA_FACEU2GF, i0,i1,i2)];
    FACEVAL[SHIFTZ] = betaz;

    /* Generate physically meaningful speeds */
    auxevol_gfs[IDX4S(VALENCIAV_RU0GF, i0,i1,i2)] = -6.609724564008164e-01;//(double)rand()/RAND_MAX*2.0-1.0;
    auxevol_gfs[IDX4S(VALENCIAV_RU1GF, i0,i1,i2)] = 5.872129306790587e-01;//(double)rand()/RAND_MAX*2.0-1.0;
    auxevol_gfs[IDX4S(VALENCIAV_RU2GF, i0,i1,i2)] = -2.394409383043848e-01;//(double)rand()/RAND_MAX*2.0-1.0;
    /* Superluminal speeds for testing */
    /*auxevol_gfs[IDX4S(VALENCIAV_RU0GF, i0,i1,i2)] = 1.0+(double)rand()/RAND_MAX*9.0;
    auxevol_gfs[IDX4S(VALENCIAV_RU1GF, i0,i1,i2)] = 1.0+(double)rand()/RAND_MAX*9.0;
    auxevol_gfs[IDX4S(VALENCIAV_RU2GF, i0,i1,i2)] = 1.0+(double)rand()/RAND_MAX*9.0;*/
    Ur[VX] = alpha*auxevol_gfs[IDX4S(VALENCIAV_RU0GF, i0,i1,i2)]-betax;
    Ur[VY] = alpha*auxevol_gfs[IDX4S(VALENCIAV_RU1GF, i0,i1,i2)]-betay;
    Ur[VZ] = alpha*auxevol_gfs[IDX4S(VALENCIAV_RU2GF, i0,i1,i2)]-betaz;

    auxevol_gfs[IDX4S(B_RU0GF, i0,i1,i2)] = 9.999999999999999e-01;//(double)rand()/RAND_MAX*2.0-1.0;
    Ur[BX_CENTER] = auxevol_gfs[IDX4S(B_RU0GF, i0,i1,i2)];
    auxevol_gfs[IDX4S(B_RU1GF, i0,i1,i2)] = 1.732050807568877e+00;//(double)rand()/RAND_MAX*2.0-1.0;
    Ur[BY_CENTER] = auxevol_gfs[IDX4S(B_RU1GF, i0,i1,i2)];
    // Set Bz to enforce orthogonality of u and B; lower index on drift v^i, then set
    // Bz = -(ux*Bx+uy*By)/uz
    REAL ux,uy,uz;
    ux = gammaDD00*(Ur[VX]+betax) + gammaDD01*(Ur[VY]+betay) + gammaDD02*(Ur[VZ]+betaz);
    uy = gammaDD01*(Ur[VX]+betax) + gammaDD11*(Ur[VY]+betay) + gammaDD12*(Ur[VZ]+betaz);
    uz = gammaDD02*(Ur[VX]+betax) + gammaDD12*(Ur[VY]+betay) + gammaDD22*(Ur[VZ]+betaz);
    auxevol_gfs[IDX4S(B_RU2GF, i0,i1,i2)] = 1.485468574153978e+00;//-(ux*Ur[BX_CENTER] + uy*Ur[BY_CENTER])/uz;
    Ur[BZ_CENTER] = auxevol_gfs[IDX4S(B_RU2GF, i0,i1,i2)];

    /* Generate physically meaningful speeds */
    auxevol_gfs[IDX4S(VALENCIAV_LU0GF, i0,i1,i2)] = -6.609724564008164e-01;//(double)rand()/RAND_MAX*2.0-1.0;
    auxevol_gfs[IDX4S(VALENCIAV_LU1GF, i0,i1,i2)] = 5.872129306790587e-01;//(double)rand()/RAND_MAX*2.0-1.0;
    auxevol_gfs[IDX4S(VALENCIAV_LU2GF, i0,i1,i2)] = -2.394409383043848e-01;//(double)rand()/RAND_MAX*2.0-1.0;
    /* Superluminal speeds for testing */
    /*auxevol_gfs[IDX4S(VALENCIAV_LU0GF, i0,i1,i2)] = 1.0+(double)rand()/RAND_MAX*9.0;
    auxevol_gfs[IDX4S(VALENCIAV_LU1GF, i0,i1,i2)] = 1.0+(double)rand()/RAND_MAX*9.0;
    auxevol_gfs[IDX4S(VALENCIAV_LU2GF, i0,i1,i2)] = 1.0+(double)rand()/RAND_MAX*9.0;*/
    Ul[VX] = alpha*auxevol_gfs[IDX4S(VALENCIAV_LU0GF, i0,i1,i2)]-betax;
    Ul[VY] = alpha*auxevol_gfs[IDX4S(VALENCIAV_LU1GF, i0,i1,i2)]-betay;
    Ul[VZ] = alpha*auxevol_gfs[IDX4S(VALENCIAV_LU2GF, i0,i1,i2)]-betaz;

    auxevol_gfs[IDX4S(B_LU0GF, i0,i1,i2)] = 9.999999999999999e-01;//(double)rand()/RAND_MAX*2.0-1.0;
    Ul[BX_CENTER] = auxevol_gfs[IDX4S(B_LU0GF, i0,i1,i2)];
    auxevol_gfs[IDX4S(B_LU1GF, i0,i1,i2)] = 1.732050807568877e+00;//(double)rand()/RAND_MAX*2.0-1.0;
    Ul[BY_CENTER] = auxevol_gfs[IDX4S(B_LU1GF, i0,i1,i2)];
    // Set Bz to enforce orthogonality of u and B; lower index on drift v^i, then set
    // Bz = -(ux*Bx+uy*By)/uz
    ux = gammaDD00*(Ul[VX]+betax) + gammaDD01*(Ul[VY]+betay) + gammaDD02*(Ul[VZ]+betaz);
    uy = gammaDD01*(Ul[VX]+betax) + gammaDD11*(Ul[VY]+betay) + gammaDD12*(Ul[VZ]+betaz);
    uz = gammaDD02*(Ul[VX]+betax) + gammaDD12*(Ul[VY]+betay) + gammaDD22*(Ul[VZ]+betaz);
    auxevol_gfs[IDX4S(B_LU2GF, i0,i1,i2)] = 1.485468574153978e+00;//-(ux*Ul[BX_CENTER] + uy*Ul[BY_CENTER])/uz;
    Ul[BZ_CENTER] = auxevol_gfs[IDX4S(B_LU2GF, i0,i1,i2)];


    printf("Valencia 3-velocity (right): %.4e, %.4e, %.4e\n",auxevol_gfs[IDX4S(VALENCIAV_RU0GF, i0,i1,i2)],auxevol_gfs[IDX4S(VALENCIAV_RU1GF, i0,i1,i2)],auxevol_gfs[IDX4S(VALENCIAV_RU2GF, i0,i1,i2)]);
    printf("Valencia 3-velocity (left):  %.4e, %.4e, %.4e\n\n",auxevol_gfs[IDX4S(VALENCIAV_LU0GF, i0,i1,i2)],auxevol_gfs[IDX4S(VALENCIAV_LU1GF, i0,i1,i2)],auxevol_gfs[IDX4S(VALENCIAV_LU2GF, i0,i1,i2)]);
    printf("Below are the numbers we care about. These are the Significant Digits of Agreement \n");
    printf("between the HLLE fluxes computed by NRPy+ and ETK. Each row represents a flux \n");
    printf("direction; each entry therein corresponds to a component of StildeD. Each pair \n");
    printf("of outputs should show at least 10 significant digits of agreement. \n\n");
    // Now, we'll run the NRPy+ and ETK functions, once in each flux_dirn.
    // We'll compare the output in-between each
    GRFFE__S_i__flux_in_dir_0(&params,auxevol_gfs);
    GRFFE__S_i__flux(0,0,0,1,Ul,Ur,FACEVAL,METRIC_LAP_PSI4,cmax,cmin,st_x_flux,st_y_flux,st_z_flux);
    printf("SDA: %.1f, %.1f, %.1f\n",1.0-log10(2.0*fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)]-st_x_flux)/(fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)])+fabs(st_x_flux))),
                                     1.0-log10(2.0*fabs(auxevol_gfs[IDX4S(STILDE_FLUXD1GF, i0,i1,i2)]-st_y_flux)/(fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)])+fabs(st_x_flux))),
                                     1.0-log10(2.0*fabs(auxevol_gfs[IDX4S(STILDE_FLUXD2GF, i0,i1,i2)]-st_z_flux)/(fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)])+fabs(st_x_flux))));
    //printf("NRPy+ Results: %.16e, %.16e, %.16e\n",auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)],auxevol_gfs[IDX4S(STILDE_FLUXD1GF, i0,i1,i2)],auxevol_gfs[IDX4S(STILDE_FLUXD2GF, i0,i1,i2)]);

    GRFFE__S_i__flux_in_dir_1(&params,auxevol_gfs);
    GRFFE__S_i__flux(0,0,0,2,Ul,Ur,FACEVAL,METRIC_LAP_PSI4,cmax,cmin,st_x_flux,st_y_flux,st_z_flux);
    printf("SDA: %.1f, %.1f, %.1f\n",1.0-log10(2.0*fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)]-st_x_flux)/(fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)])+fabs(st_x_flux))),
                                     1.0-log10(2.0*fabs(auxevol_gfs[IDX4S(STILDE_FLUXD1GF, i0,i1,i2)]-st_y_flux)/(fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)])+fabs(st_x_flux))),
                                     1.0-log10(2.0*fabs(auxevol_gfs[IDX4S(STILDE_FLUXD2GF, i0,i1,i2)]-st_z_flux)/(fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)])+fabs(st_x_flux))));
    //printf("NRPy+ Results: %.16e, %.16e, %.16e\n",auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)],auxevol_gfs[IDX4S(STILDE_FLUXD1GF, i0,i1,i2)],auxevol_gfs[IDX4S(STILDE_FLUXD2GF, i0,i1,i2)]);

    GRFFE__S_i__flux_in_dir_2(&params,auxevol_gfs);
    GRFFE__S_i__flux(0,0,0,3,Ul,Ur,FACEVAL,METRIC_LAP_PSI4,cmax,cmin,st_x_flux,st_y_flux,st_z_flux);
    printf("SDA: %.1f, %.1f, %.1f\n",1.0-log10(2.0*fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)]-st_x_flux)/(fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)])+fabs(st_x_flux))),
                                     1.0-log10(2.0*fabs(auxevol_gfs[IDX4S(STILDE_FLUXD1GF, i0,i1,i2)]-st_y_flux)/(fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)])+fabs(st_x_flux))),
                                     1.0-log10(2.0*fabs(auxevol_gfs[IDX4S(STILDE_FLUXD2GF, i0,i1,i2)]-st_z_flux)/(fabs(auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)])+fabs(st_x_flux))));
    //printf("NRPy+ Results: %.16e, %.16e, %.16e\n",auxevol_gfs[IDX4S(STILDE_FLUXD0GF, i0,i1,i2)],auxevol_gfs[IDX4S(STILDE_FLUXD1GF, i0,i1,i2)],auxevol_gfs[IDX4S(STILDE_FLUXD2GF, i0,i1,i2)]);

    printf("Note that in the case of very high velocities, numerical error will accumulate \n");
    printf("and reduce agreement significantly due to a catastrophic cancellation. \n\n");

    //printf("ETK   Results: %.16e, %.16e, %.16e\n\n",st_x_flux,st_y_flux,st_z_flux);
}


Overwriting Validation/Stilde_flux_unit_test.C


<a id='compile_run'></a>

## Step 2.a: Compile and run the code to validate the output \[Back to [top](#toc)\]

$$\label{compile_run}$$

And now, we will compile and run the C code. We also make python calls to time how long each of these steps takes.

In [14]:
import cmdline_helper as cmd
import time
print("Now compiling, should take ~2 seconds...\n")
start = time.time()
cmd.C_compile(os.path.join(out_dir,"Stilde_flux_unit_test.C"), os.path.join(out_dir,"Stilde_flux_unit_test"))
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")

# os.chdir(out_dir)
print("Now running...\n")
start = time.time()
# cmd.Execute(os.path.join("Stilde_flux_unit_test"))
!./Validation/Stilde_flux_unit_test
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")
# os.chdir(os.path.join("../"))


Now compiling, should take ~2 seconds...

Compiling executable...
Executing `gcc -Ofast -fopenmp -march=native -funroll-loops Validation/Stilde_flux_unit_test.C -o Validation/Stilde_flux_unit_test -lm`...
Finished executing in 0.6145997047424316 seconds.
Finished compilation.
Finished in 0.6322972774505615 seconds.


Now running...

Valencia 3-velocity (right): -6.6097e-01, 5.8721e-01, -2.3944e-01
Valencia 3-velocity (left):  -6.6097e-01, 5.8721e-01, -2.3944e-01

Below are the numbers we care about. These are the Significant Digits of Agreement 
between the HLLE fluxes computed by NRPy+ and ETK. Each row represents a flux 
direction; each entry therein corresponds to a component of StildeD. Each pair 
of outputs should show at least 10 significant digits of agreement. 

SDA: 16.3, 16.3, 16.8
SDA: 16.4, 16.3, 16.7
SDA: 16.2, 16.0, 16.5
Note that in the case of very high velocities, numerical error will accumulate 
and reduce agreement significantly due to a catastrophic cancellation. 



<a id='latex_pdf_output'></a>

# Step 3: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Start_to_Finish_UnitTest-GiRaFFE_NRPy-Stilde_flux.pdf](Tutorial-Start_to_Finish_UnitTest-GiRaFFE_NRPy-Stilde_flux.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [8]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Start_to_Finish_UnitTest-GiRaFFE_NRPy-Stilde_flux")

This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
